In [53]:
from faker import Faker
import pandas as pd
import numpy as np
import os


## Inventory Data

In [54]:
inventory_df = pd.read_csv('./data/inventory_data.csv')
inventory_df

,Date,Item_ID,Item_Type,Item_Name,Current_Stock,Min_Required,Max_Capacity,Unit_Cost,Avg_Usage_Per_Day,Restock_Lead_Time,Vendor_ID
0,2024-10-01,105,Consumable,Ventilator,1542,264,1018,4467.55,108,17,V001
1,2024-10-02,100,Equipment,Ventilator,2487,656,3556,5832.29,55,12,V001
2,2024-10-03,103,Equipment,Surgical Mask,2371,384,5562,16062.98,470,6,V001
3,2024-10-04,103,Consumable,Surgical Mask,2038,438,1131,744.10,207,15,V002
4,2024-10-05,107,Equipment,IV Drip,2410,338,1013,15426.53,158,12,V003
...,...,...,...,...,...,...,...,...,...,...,...
495,2026-02-08,108,Consumable,Ventilator,4072,776,3496,15934.20,257,6,V002
496,2026-02-09,100,Equipment,X-ray Machine,997,239,2186,5863.88,108,16,V002
497,2026-02-10,107,Equipment,Surgical Mask,4193,304,4213,90.30,419,27,V001
498,2026-02-11,106,Consumable,IV Drip,822,186,2385,5284.45,382,6,V003


In [55]:
## First, generate unique items

supplies_dict = {
    'Ventilator': {
        'item_id': 100,
        'item_type': 'Equipment',
        'unit_cost': (1500000, 3200000),
        'min_required': (3, 15),
        'max_capacity': (25, 50),
        'avg_usage_per_day': (1, 5),
        'restock_lead_time': (15, 30),
    },
    'X-Ray-Machine': {
        'item_id': 101,
        'item_type': 'Equipment',
        'unit_cost': (2500000, 5000000),
        'min_required': (1, 5),
        'max_capacity': (10, 20),
        'avg_usage_per_day': (0, 1),
        'restock_lead_time': (20, 45),
    },
    'Defibrilator': {
        'item_id': 102,
        'item_type': 'Equipment',
        'unit_cost': (300000, 750000),
        'min_required': (1, 5),
        'max_capacity': (8, 15),
        'avg_usage_per_day': (0, 1),
        'restock_lead_time': (20, 40),
    },
    'Wheelchair': {
        'item_id': 103,
        'item_type': 'Equipment',
        'unit_cost': (5000, 25000), # Manual vs Electric Wheelchairs
        'min_required': (5, 20),
        'max_capacity': (30, 100),
        'avg_usage_per_day': (0, 5),
        'restock_lead_time': (7, 20),
    },
    'Hospital Bed': {
        'item_id': 104,
        'item_type': 'Equipment',
        'unit_cost': (12000, 60000),
        'min_required': (25, 100),
        'max_capacity': (150, 300),
        'avg_usage_per_day': (0, 3),
        'restock_lead_time': (15, 30),
    },
    'ECG Machine': {
        'item_id': 105,
        'item_type': 'Equipment',
        'unit_cost': (100000, 400000),
        'min_required': (2, 5),
        'max_capacity': (10, 15),
        'avg_usage_per_day': (0, 3),
        'restock_lead_time': (15, 30),
    },
    'MRI Scanner': {
        'item_id': 106,
        'item_type': 'Equipment',
        'unit_cost': (25000000, 50000000),
        'min_required': (1, 2),
        'max_capacity': (2, 5),
        'avg_usage_per_day': (0, 1),
        'restock_lead_time': (30, 60),
    },
    'Infusion Pump': {
        'item_id': 107,
        'item_type': 'Equipment',
        'unit_cost': (80000, 250000),
        'min_required': (3, 10),
        'max_capacity': (15, 30),
        'avg_usage_per_day': (5, 15),
        'restock_lead_time': (10, 20),
    },
    'Gloves': {
        'item_id': 200,
        'item_type': 'Consumable',
        'unit_cost': (5, 15),
        'min_required': (1000, 3000),
        'max_capacity': (5000, 10000),
        'avg_usage_per_day': (200, 1000),
        'restock_lead_time': (5, 12),
    },
    'IV Drip': {
        'item_id': 201,
        'item_type': 'Consumable',
        'unit_cost': (120, 350),
        'min_required': (300, 1000),
        'max_capacity': (2000, 5000),
        'avg_usage_per_day': (50, 200),
        'restock_lead_time': (5, 10),
    },
    'Surgical Mask': {
        'item_id': 202,
        'item_type': 'Consumable',
        'unit_cost': (3, 8),
        'min_required': (3000, 10000),
        'max_capacity': (15000, 50000),
        'avg_usage_per_day': (500, 3000),
        'restock_lead_time': (3, 10),
    },
    'Gown': {
        'item_id': 203,
        'item_type': 'Consumable',
        'unit_cost': (80, 250),
        'min_required': (500, 2000),
        'max_capacity': (3000, 8000),
        'avg_usage_per_day': (100, 500),
        'restock_lead_time': (5, 12),
    },
    'Face Shield': {
        'item_id': 204,
        'item_type': 'Consumable',
        'unit_cost': (50, 120),
        'min_required': (500, 2000),
        'max_capacity': (4000, 10000),
        'avg_usage_per_day': (100, 800),
        'restock_lead_time': (3, 8),
    },
    'Syringe': {
        'item_id': 205,
        'item_type': 'Consumable',
        'unit_cost': (7, 35),
        'min_required': (1500, 5000),
        'max_capacity': (8000, 20000),
        'avg_usage_per_day': (200, 2000),
        'restock_lead_time': (5, 12),
    },
    'Bandages': {
        'item_id': 206,
        'item_type': 'Consumable',
        'unit_cost': (10, 50),
        'min_required': (1000, 4000),
        'max_capacity': (6000, 12000),
        'avg_usage_per_day': (300, 1500),
        'restock_lead_time': (3, 10),
    },
    'Blood Bags': {
        'item_id': 207,
        'item_type': 'Consumable',
        'unit_cost': (400, 1500),
        'min_required': (300, 1000),
        'max_capacity': (1500, 5000),
        'avg_usage_per_day': (20, 150),
        'restock_lead_time': (5, 15),
    },
    'Antiseptic Solution': {
        'item_id': 208,
        'item_type': 'Consumable',
        'unit_cost': (200, 700),
        'min_required': (500, 2000),
        'max_capacity': (3000, 8000),
        'avg_usage_per_day': (100, 500),
        'restock_lead_time': (5, 12),
    },
    'Cotton Rolls': {
        'item_id': 209,
        'item_type': 'Consumable',
        'unit_cost': (50, 150),
        'min_required': (300, 1000),
        'max_capacity': (2000, 6000),
        'avg_usage_per_day': (50, 300),
        'restock_lead_time': (3, 10),
    },
}

In [56]:
item_var_dict = {}

for key, value in supplies_dict.items():
    # Generate 5 different item variations for 5 different vendors
    for i in range(5):
        item_name = key
        item_id = value['item_id']
        item_type_arr = value['item_type']
        unit_cost = np.random.randint(value['unit_cost'][0], value['unit_cost'][1] + 1)
        vendor_id = f'V{str(i+1).zfill(3)}'

        item_var_dict[f'{item_id}-{vendor_id}'] = {
            'item_name': item_name,
            'item_id': item_id,
            'item_type': item_type_arr,
            'unit_cost': unit_cost,
            'vendor_id': vendor_id,
        }

item_var_dict

{'100-V001': {'item_name': 'Ventilator',
  'item_id': 100,
  'item_type': 'Equipment',
  'unit_cost': 1999337,
  'vendor_id': 'V001'},
 '100-V002': {'item_name': 'Ventilator',
  'item_id': 100,
  'item_type': 'Equipment',
  'unit_cost': 3142549,
  'vendor_id': 'V002'},
 '100-V003': {'item_name': 'Ventilator',
  'item_id': 100,
  'item_type': 'Equipment',
  'unit_cost': 2900485,
  'vendor_id': 'V003'},
 '100-V004': {'item_name': 'Ventilator',
  'item_id': 100,
  'item_type': 'Equipment',
  'unit_cost': 1879046,
  'vendor_id': 'V004'},
 '100-V005': {'item_name': 'Ventilator',
  'item_id': 100,
  'item_type': 'Equipment',
  'unit_cost': 2232401,
  'vendor_id': 'V005'},
 '101-V001': {'item_name': 'X-Ray-Machine',
  'item_id': 101,
  'item_type': 'Equipment',
  'unit_cost': 2798274,
  'vendor_id': 'V001'},
 '101-V002': {'item_name': 'X-Ray-Machine',
  'item_id': 101,
  'item_type': 'Equipment',
  'unit_cost': 4180254,
  'vendor_id': 'V002'},
 '101-V003': {'item_name': 'X-Ray-Machine',
  'it

In [57]:
fake = Faker()
date_list = pd.date_range(start='2022-01-01', end='2024-12-31', freq='D').strftime('%Y-%m-%d').tolist()

# Data dictionary (to pass to df constructor)
date_arr = []
item_id_arr = []
item_type_arr = []
item_name_arr = []
unit_cost_arr = []
min_required_arr = []
max_capacity_arr = []
current_stock_arr = []
avg_usage_per_day_arr = []
restock_lead_time_arr = []
vendor_id_arr = []

for key, value in item_var_dict.items():
    # Generate around 30 - 50 random instances of a particular item
    for _ in range(np.random.randint(30, 51)):
        date = fake.random_element(date_list)
        item_id = value['item_id']
        item_type = value['item_type']
        item_name = value['item_name']
        unit_cost = value['unit_cost']
        min_required = np.random.randint(supplies_dict[item_name]['min_required'][0], supplies_dict[item_name]['min_required'][1] + 1)
        max_capacity = np.random.randint(supplies_dict[item_name]['max_capacity'][0], supplies_dict[item_name]['max_capacity'][1] + 1)
        current_stock = np.random.randint(min_required, max_capacity + 1)
        avg_usage_per_day = np.random.randint(supplies_dict[item_name]['avg_usage_per_day'][0], supplies_dict[item_name]['avg_usage_per_day'][1] + 1)
        restock_lead_time = np.random.randint(supplies_dict[item_name]['restock_lead_time'][0], supplies_dict[item_name]['restock_lead_time'][1] + 1)
        vendor_id = value['vendor_id']

        date_arr.append(date)
        item_id_arr.append(item_id)
        item_type_arr.append(item_type)
        item_name_arr.append(item_name)
        unit_cost_arr.append(unit_cost)
        min_required_arr.append(min_required)
        max_capacity_arr.append(max_capacity)
        current_stock_arr.append(current_stock)
        avg_usage_per_day_arr.append(avg_usage_per_day)
        restock_lead_time_arr.append(restock_lead_time)
        vendor_id_arr.append(vendor_id)

data = list(zip(date_arr, item_id_arr, item_type_arr, item_name_arr, unit_cost_arr, min_required_arr, max_capacity_arr, current_stock_arr, avg_usage_per_day_arr, restock_lead_time_arr, vendor_id_arr))

df = pd.DataFrame(data, columns=[
    'date', 'item_id', 'item_type', 'item_name', 'unit_cost', 'min_required', 'max_capacity', 'current_stock', 'avg_usage_per_day', 'restock_lead_time', 'vendor_id'
]).sort_values(by=['date']).reset_index(drop=True)

In [58]:
df.head(50)

,date,item_id,item_type,item_name,unit_cost,min_required,max_capacity,current_stock,avg_usage_per_day,restock_lead_time,vendor_id
0,2022-01-01,202,Consumable,Surgical Mask,7,8222,30476,23033,2215,9,V002
1,2022-01-01,101,Equipment,X-Ray-Machine,3863821,2,17,10,1,29,V003
2,2022-01-01,203,Consumable,Gown,102,1609,4231,2073,281,8,V005
3,2022-01-01,206,Consumable,Bandages,42,1732,9898,3921,909,8,V003
4,2022-01-02,107,Equipment,Infusion Pump,125406,9,28,11,8,10,V004
5,2022-01-02,100,Equipment,Ventilator,2232401,8,40,26,5,27,V005
6,2022-01-03,105,Equipment,ECG Machine,366194,4,12,10,1,26,V003
7,2022-01-03,203,Consumable,Gown,81,1508,7165,1986,335,9,V004
8,2022-01-03,206,Consumable,Bandages,41,1087,11026,5091,625,10,V005
9,2022-01-03,100,Equipment,Ventilator,2900485,6,31,10,5,17,V003


In [59]:
file_name = './data/inventory_data_v2.csv'

if not os.path.exists(file_name):
    df.to_csv(file_name, index=False)
    print('File saved successfully!')
else:
    print('File already exists!')

File saved successfully!


## Patient Data

In [60]:
patient_df = pd.read_csv('./data/patient_data.csv')
patient_df

,Patient_ID,Admission_Date,Discharge_Date,Primary_Diagnosis,Procedure_Performed,Room_Type,Bed_Days,Supplies_Used,Equipment_Used,Staff_Needed
0,P001,2024-10-06 05:30:28,2024-10-23 01:11:34,Diabetes,Appendectomy,General Ward,2,"Gloves, IV",Surgical Table,2 Surgeons
1,P002,2024-10-24 11:07:58,2024-10-15 05:16:54,Fracture,Appendectomy,ICU,10,"Gown, IV",MRI Machine,1 Nurse
2,P003,2024-10-22 21:43:43,2024-10-24 10:56:30,Fracture,Chest X-ray,ICU,10,"Gloves, IV",X-ray Machine,"1 Nurse, 1 Doctor"
3,P004,2024-10-05 17:04:05,2024-10-30 14:10:01,Diabetes,Chest X-ray,ICU,11,"Gloves, IV",X-ray Machine,1 Nurse
4,P005,2024-10-21 17:04:00,2024-10-08 15:53:22,Appendicitis,MRI,ICU,2,"Gloves, IV",X-ray Machine,2 Surgeons
...,...,...,...,...,...,...,...,...,...,...
495,P496,2024-10-01 12:54:33,2024-10-06 14:31:39,Diabetes,MRI,ICU,13,"Mask, Gown",Surgical Table,"1 Nurse, 1 Doctor"
496,P497,2024-10-22 07:37:10,2024-10-13 12:31:40,Appendicitis,Appendectomy,ICU,14,"Gloves, IV",MRI Machine,"1 Nurse, 1 Doctor"
497,P498,2024-10-28 03:12:20,2024-10-10 08:11:18,Pneumonia,MRI,General Ward,13,"Mask, Gown",X-ray Machine,2 Surgeons
498,P499,2024-10-04 22:02:01,2024-10-12 03:45:09,Diabetes,Chest X-ray,General Ward,1,"Gown, IV",Surgical Table,"1 Nurse, 1 Doctor"


In [61]:
# Supplies Used Item Mapping
supplies_dict = {}

for i in patient_df['Supplies_Used']:
    item_arr = i.split(', ')
    for item in item_arr:
        if item not in supplies_dict:
            supplies_dict[item] = 1
        elif item in supplies_dict:
            supplies_dict[item] += 1
        
supplies_dict

{'Gloves': 169, 'IV': 339, 'Gown': 331, 'Mask': 161}

In [62]:
# Equipment Used Item Mapping
equipment_dict = {}

for i in patient_df['Equipment_Used']:
    item_arr = i.split(', ')
    for item in item_arr:
        if item not in equipment_dict:
            equipment_dict[item] = 1
        elif item in equipment_dict:
            equipment_dict[item] += 1
        
equipment_dict

{'Surgical Table': 166, 'MRI Machine': 161, 'X-ray Machine': 173}

In [63]:
# Rename the keys in the dictionary to standardized item names
try:
    supplies_dict['IV Drip'] = supplies_dict.pop('IV')
    supplies_dict['Surgical Mask'] = supplies_dict.pop('Mask')
    equipment_dict['X-Ray-Machine'] = equipment_dict.pop('X-ray Machine')
    equipment_dict['MRI Scanner'] = equipment_dict.pop('MRI Machine')
except KeyError:
    pass

In [64]:
# EDA of 'primary_diagnosis'
patient_df['Primary_Diagnosis'].value_counts()

Primary_Diagnosis
Fracture        133
Diabetes        131
Appendicitis    120
Pneumonia       116
Name: count, dtype: int64

In [65]:
# EDA of 'procedure_performed'
patient_df['Procedure_Performed'].value_counts()

Procedure_Performed
Chest X-ray     138
Appendectomy    131
MRI             131
Blood Test      100
Name: count, dtype: int64

### Synthetic Patient Data Generation

In [66]:
diagnosis_dict = {
    'Pneumonia': {
        'Chest X-Ray': {
            'supplies_used': ['Surgical Mask', 'Gloves', 'Gown', 'Face Shield'],
            'equipment_used': ['X-Ray Machine']
        },
        'Blood Test': {
            'supplies_used': ['Syringe', 'Cotton Rolls', 'Gloves'],
            'equipment_used': []
        },
        'IV Therapy': {
            'supplies_used': ['IV Drip', 'Syringe', 'Gloves', 'Antiseptic Solution'],
            'equipment_used': ['Infusion Pump']
        },
        'Oxygen Therapy': {
            'supplies_used': ['Surgical Mask', 'Face Shield', 'Gloves'],
            'equipment_used': ['Ventilator']
        }
    },
    'Appendicitis': {
        'Appendectomy': {
            'supplies_used': ['Gloves', 'Gown', 'Surgical Mask', 'Antiseptic Solution', 'Bandages', 'IV Drip'],
            'equipment_used': ['Hospital Bed', 'Infusion Pump']
        },
        'Blood Test': {
            'supplies_used': ['Syringe', 'Cotton Rolls', 'Gloves'],
            'equipment_used': []
        }
    },
    'Fracture': {
        'X-Ray': {
            'supplies_used': ['Surgical Mask', 'Gloves', 'Gown', 'Face Shield'],
            'equipment_used': ['X-Ray Machine']
        },
        'Casting': {
            'supplies_used': ['Bandages', 'Cotton Rolls', 'Gloves', 'Antiseptic Solution'],
            'equipment_used': []
        },
        'Pain Management': {
            'supplies_used': ['Syringe', 'IV Drip', 'Gloves'],
            'equipment_used': ['Hospital Bed', 'Infusion Pump']
        }
    },
    'Myocardial Infarction': {
        'ECG': {
            'supplies_used': ['Gloves'],
            'equipment_used': ['ECG Machine']
        },
        'Defibrillation': {
            'supplies_used': ['Gloves', 'Surgical Mask', 'Antiseptic Solution'],
            'equipment_used': ['Defibrillator']
        },
        'IV Therapy': {
            'supplies_used': ['IV Drip', 'Syringe', 'Gloves', 'Blood Bags'],
            'equipment_used': ['Infusion Pump']
        }
    },
    'Diabetes Complications': {
        'Blood Test': {
            'supplies_used': ['Syringe', 'Cotton Rolls', 'Gloves'],
            'equipment_used': []
        },
        'IV Therapy': {
            'supplies_used': ['IV Drip', 'Syringe', 'Gloves', 'Blood Bags'],
            'equipment_used': ['Infusion Pump']
        },
        'Wound Dressing': {
            'supplies_used': ['Bandages', 'Gloves', 'Antiseptic Solution', 'Cotton Rolls'],
            'equipment_used': []
        }
    },
    'Stroke': {
        'MRI': {
            'supplies_used': ['Surgical Mask', 'Gloves', 'Gown', 'Face Shield'],
            'equipment_used': ['MRI Scanner']
        },
        'Blood Test': {
            'supplies_used': ['Syringe', 'Cotton Rolls', 'Gloves'],
            'equipment_used': []
        },
        'Physical Therapy': {
            'supplies_used': ['Gloves', 'Face Shield', 'Gown'],
            'equipment_used': ['Hospital Bed', 'Wheelchair']
        }
    },
    'Sepsis': {
        'Blood Test': {
            'supplies_used': ['Syringe', 'Cotton Rolls', 'Gloves', 'Blood Bags'],
            'equipment_used': []
        },
        'IV Antibiotic Therapy': {
            'supplies_used': ['IV Drip', 'Gloves', 'Antiseptic Solution'],
            'equipment_used': ['Infusion Pump']
        }
    },
    'Severe Burns': {
        'Wound Cleaning & Dressing': {
            'supplies_used': ['Antiseptic Solution', 'Bandages', 'Gloves', 'Cotton Rolls'],
            'equipment_used': []
        },
        'Skin Grafting Surgery': {
            'supplies_used': ['Gown', 'Surgical Mask', 'Gloves'],
            'equipment_used': ['Hospital Bed']
        },
        'IV Fluid Therapy': {
            'supplies_used': ['IV Drip', 'Syringe', 'Gloves'],
            'equipment_used': ['Infusion Pump']
        }
    },
    'Traumatic Brain Injury': {
        'MRI': {
            'supplies_used': ['Surgical Mask', 'Gloves', 'Gown', 'Face Shield'],
            'equipment_used': ['MRI Scanner']
        },
        'Ventilation Support': {
            'supplies_used': ['Surgical Mask', 'Face Shield', 'Gloves'],
            'equipment_used': ['Ventilator']
        },
        'Pain Management': {
            'supplies_used': ['Syringe', 'IV Drip', 'Gloves'],
            'equipment_used': ['Hospital Bed', 'Infusion Pump']
        }
    },
    'COVID-19': {
        'Oxygen Therapy': {
            'supplies_used': ['Surgical Mask', 'Face Shield', 'Gloves', 'Gown'],
            'equipment_used': ['Ventilator', 'Hospital Bed']
        },
        'Blood Test': {
            'supplies_used': ['Syringe', 'Cotton Rolls', 'Gloves'],
            'equipment_used': []
        },
        'Chest X-Ray': {
            'supplies_used': ['Surgical Mask', 'Gloves', 'Gown', 'Face Shield'],
            'equipment_used': ['X-Ray Machine']
        }
    }
}

In [67]:
diagnosis_id_arr = []
date_arr = []
primary_diagnosis_arr = []
procedure_performed_arr = []
item_used_arr = []
item_type_arr = []

diagnosis_id_counter = 1

date_list = pd.date_range(start='2022-01-01', end='2024-12-31', freq='D').strftime('%Y-%m-%d').tolist()

for diagnosis in diagnosis_dict:
    for _ in range(np.random.randint(100, 500)):
        diagnosis_id = diagnosis_id_counter
        for procedure in diagnosis_dict[diagnosis]:
            for item in diagnosis_dict[diagnosis][procedure]['supplies_used']:
                diagnosis_id_arr.append(str(diagnosis_id).zfill(5))
                date_arr.append(fake.random_element(date_list))
                primary_diagnosis_arr.append(diagnosis)
                procedure_performed_arr.append(procedure)
                item_used_arr.append(item)
                item_type_arr.append('Consumable')
            for item in diagnosis_dict[diagnosis][procedure]['equipment_used']:
                diagnosis_id_arr.append(str(diagnosis_id).zfill(5))
                date_arr.append(fake.random_element(date_list))
                primary_diagnosis_arr.append(diagnosis)
                procedure_performed_arr.append(procedure)
                item_used_arr.append(item)
                item_type_arr.append('Equipment')
        diagnosis_id_counter += 1

In [68]:
print(len(diagnosis_id_arr))
print(len(primary_diagnosis_arr))
print(len(procedure_performed_arr))
print(len(item_used_arr))
print(len(item_type_arr))

33214
33214
33214
33214
33214


In [69]:
# Data dictionary (to pass to df constructor)
data = {
    'diagnosis_id': diagnosis_id_arr,
    'date': date_arr,
    'primary_diagnosis': primary_diagnosis_arr,
    'procedure_performed': procedure_performed_arr,
    'item_used': item_used_arr,
    'item_type': item_type_arr
}

patient_df_v2 = pd.DataFrame(data)
patient_df_v2

,diagnosis_id,date,primary_diagnosis,procedure_performed,item_used,item_type
0,00001,2022-09-08,Pneumonia,Chest X-Ray,Surgical Mask,Consumable
1,00001,2022-10-23,Pneumonia,Chest X-Ray,Gloves,Consumable
2,00001,2022-07-06,Pneumonia,Chest X-Ray,Gown,Consumable
3,00001,2022-04-11,Pneumonia,Chest X-Ray,Face Shield,Consumable
4,00001,2023-11-08,Pneumonia,Chest X-Ray,X-Ray Machine,Equipment
...,...,...,...,...,...,...
33209,02692,2023-08-29,COVID-19,Chest X-Ray,Surgical Mask,Consumable
33210,02692,2024-02-16,COVID-19,Chest X-Ray,Gloves,Consumable
33211,02692,2024-03-26,COVID-19,Chest X-Ray,Gown,Consumable
33212,02692,2023-04-21,COVID-19,Chest X-Ray,Face Shield,Consumable


In [70]:
file_name = './data/patient_data_v2.csv'

if not os.path.exists(file_name):
    patient_df_v2.to_csv(file_name, index=False)
    print('File saved successfully!')
else:
    print('File already exists!')

File saved successfully!
